# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
gkey = os.environ.get("google_maps_key")

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("../WeatherPy/data.csv")
df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Yulara,-25.24,130.99,57.20,30,0,10.29,AU,1596069696
1,1,Byron Bay,-28.65,153.62,64.40,63,0,12.75,AU,1596069696
2,2,Avarua,-21.21,-159.78,69.80,83,1,6.93,CK,1596069696
3,3,Saskylakh,71.92,114.08,57.90,59,0,3.31,RU,1596069696
4,4,Busselton,-33.65,115.33,51.01,97,82,6.69,AU,1596069527


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey)

In [4]:
locations = df[["Lat", "Lng"]]
weight = df["Humidity"]
fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights = weight, max_intensity=100, dissipating=False, point_radius=2)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_df = df.loc[(df["Max Temp"] > 70) & (df["Max Temp"] < 80) & (df["Wind Speed"]  < 10) & (df["Cloudiness"] == 0)]
ideal_df.reset_index()

,index,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,18,18,Tiznit Province,29.58,-9.50,75.72,45,0,0.96,MA,1596069698
1,51,51,Vostok,46.49,135.88,77.99,56,0,4.52,RU,1596069703
2,65,65,Irauçuba,-3.75,-39.78,75.36,65,0,4.92,BR,1596069704
3,106,106,Ol'ga,43.74,135.29,78.76,57,0,6.60,RU,1596069491
4,146,146,Camocim,-2.90,-40.84,75.90,81,0,5.57,BR,1596069713
5,165,165,Altamira,-3.20,-52.21,73.69,71,0,1.63,BR,1596069716
6,191,191,Yangi Marg`ilon,40.43,71.72,75.20,41,0,6.93,UZ,1596069719
7,244,244,Yumen,40.28,97.20,74.89,25,0,5.37,CN,1596069725
8,306,306,Bystrogorskiy,48.21,41.14,71.01,47,0,2.26,RU,1596069732
9,376,376,Mozarlândia,-14.74,-50.57,73.94,43,0,4.90,BR,1596069740


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_df
hotel_df.reset_index()
hotel_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,Tiznit Province,29.58,-9.50,75.72,45,0,0.96,MA,1596069698
51,51,Vostok,46.49,135.88,77.99,56,0,4.52,RU,1596069703
65,65,Irauçuba,-3.75,-39.78,75.36,65,0,4.92,BR,1596069704
106,106,Ol'ga,43.74,135.29,78.76,57,0,6.60,RU,1596069491
146,146,Camocim,-2.90,-40.84,75.90,81,0,5.57,BR,1596069713


In [7]:
hotel_list = []
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, city in hotel_df.iterrows():
    try:
        lat = city["Lat"]
        lng = city["Lng"]
        params = {"key": gkey, 
          "location": f"{lat},{lng}",
         "radius":5000,
         "type": "lodging",
         "keyword":"hotel"}
        response = requests.get(url, params=params).json()
        hotel_name = response["results"][0]["name"]
        hotel_list.append(hotel_name)
    except IndexError:
        hotel_list.append("No hotel found")

hotel_df["Hotel Name"] = hotel_list
hotel_df.reset_index()
hotel_df


C:\Users\flurp\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,18,Tiznit Province,29.58,-9.50,75.72,45,0,0.96,MA,1596069698,No hotel found
51,51,Vostok,46.49,135.88,77.99,56,0,4.52,RU,1596069703,No hotel found
65,65,Irauçuba,-3.75,-39.78,75.36,65,0,4.92,BR,1596069704,Pousada Blue Marlin
106,106,Ol'ga,43.74,135.29,78.76,57,0,6.60,RU,1596069491,Gostinitsa
146,146,Camocim,-2.90,-40.84,75.90,81,0,5.57,BR,1596069713,Villa del Mar Praia Hotel
165,165,Altamira,-3.20,-52.21,73.69,71,0,1.63,BR,1596069716,Xingu Praia Hotel
191,191,Yangi Marg`ilon,40.43,71.72,75.20,41,0,6.93,UZ,1596069719,Hotel Mehmon Saroy
244,244,Yumen,40.28,97.20,74.89,25,0,5.37,CN,1596069725,No hotel found
306,306,Bystrogorskiy,48.21,41.14,71.01,47,0,2.26,RU,1596069732,No hotel found
376,376,Mozarlândia,-14.74,-50.57,73.94,43,0,4.90,BR,1596069740,Hotel Santa Mônica


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations=locations, info_box_content= hotel_info )
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))